In [ ]:
import numpy as np
import pandas as pd
import re
import math
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load Parameters of Elements

In [ ]:
param = pd.read_csv("/content/drive/MyDrive/Mendeley Dataset/New_Folder/final_dataset_parameters.csv")
param.head()


,Unnamed: 0,mp,enthalphy,en,ar
0,Ag,961.780,11.30,1.93,1.72
1,Al,660.323,10.70,1.61,1.84
2,Au,1064.180,12.50,2.54,1.66
3,B,2075.000,50.00,2.04,1.92
4,Be,1287.000,7.95,1.57,1.53


In [ ]:
param.columns

Index(['Unnamed: 0', 'mp', 'enthalphy', 'en', 'ar'], dtype='object')

In [ ]:
param.set_index('Unnamed: 0', drop=True, inplace=True)
pm = param.to_dict(orient="index")

In [ ]:
pm

{'Ag': {'ar': 1.72, 'en': 1.93, 'enthalphy': 11.3, 'mp': 961.78},
 'Al': {'ar': 1.84, 'en': 1.61, 'enthalphy': 10.7, 'mp': 660.323},
 'Au': {'ar': 1.66, 'en': 2.54, 'enthalphy': 12.5, 'mp': 1064.18},
 'B': {'ar': 1.92, 'en': 2.04, 'enthalphy': 50.0, 'mp': 2075.0},
 'Ba': {'ar': 2.68, 'en': 0.89, 'enthalphy': 8.0, 'mp': 727.0},
 'Be': {'ar': 1.53, 'en': 1.57, 'enthalphy': 7.95, 'mp': 1287.0},
 'C': {'ar': 1.7, 'en': 2.55, 'enthalphy': 105.0, 'mp': 3550.0},
 'Ca': {'ar': 2.31, 'en': 1.0, 'enthalphy': 8.53, 'mp': 842.0},
 'Ce': {'ar': 2.35, 'en': 1.12, 'enthalphy': 5.5, 'mp': 798.0},
 'Co': {'ar': 1.92, 'en': 1.88, 'enthalphy': 16.2, 'mp': 1495.0},
 'Cr': {'ar': 1.89, 'en': 1.66, 'enthalphy': 20.5, 'mp': 1907.0},
 'Cu': {'ar': 1.4, 'en': 1.9, 'enthalphy': 13.1, 'mp': 1084.62},
 'Dy': {'ar': 2.29, 'en': 1.22, 'enthalphy': 11.1, 'mp': 1412.0},
 'Er': {'ar': 2.35, 'en': 1.24, 'enthalphy': 19.9, 'mp': 1529.0},
 'Fe': {'ar': 1.94, 'en': 1.83, 'enthalphy': 13.8, 'mp': 1538.0},
 'Ga': {'ar': 1.8

In [ ]:
len(pm)

54

In [ ]:
def diff(x):

  #making ranges for each atom
  ranges = {}
  for i in x:
    ranges[i] = 0.88*pm[i]['ar']
  
  #compiling scoring matrix
  score = {}
  
  for i in x:
    dct = {}
    for j in x:
      if pm[i]['ar']<ranges[j]:
        dct[j] = -1
      elif pm[i]['ar']>pm[j]['ar']:
        dct[j] = 1
      else :
        dct[j] = 0
          
    score[i] = dct
    
  b = []
  sm = []
  #separating into big and small based on scoring matrix
  for i in score:
    sum = 0
    for j in score[i]:
      sum = sum+score[i][j]
      
    if(sum>0):
      b.append(i)
    else: 
      sm.append(i)
      
  if len(b)==0 or len(sm)==0:
    print(score)
  return b,sm

In [ ]:
#finds the paramater deltaE
def electro(all,x):
  k =0 #summation of product of composition and electro negativity
  p = 0 #summation of compositions

  for i in all:
    if i in x:
      p = p+all[i]
  for i in all:
    if i in x:
      k = k+all[i]*pm[i]['en']
  
  return k/p

In [ ]:
#finds the paramater deltaD
def comps(all,x):
  k =0 #summation of product of composition and atomic radii
  p = 0 #summation of compositions

  for i in all:
    if i in x:
      p = p+all[i]
  for i in all:
    if i in x:
      k = k+all[i]*pm[i]['ar']
  
  return k/p

In [ ]:
def prepare_params(x):
  #Seperating atoms from composition
  s = re.sub(r'[^\w\s]','',x)
  s = re.sub('\d',' ',s)
  els = np.array([i for i in s.split(' ') if i in pm]) # elements list
  # print('\nElements in BMG are : ', els)
  
  comp = re.findall(r"[-+]?\d*\.\d+|\d+",x)
  comp = [float(i) for i in comp]
  # print('Compositions: ',comp)
  
  dcomp = dict(zip(els,comp))
  smix = 0
  hmix = 0

  for i in dcomp:
    smix = smix + (dcomp[i]/100)*(math.log((dcomp[i]/100)))
    hmix = hmix + (dcomp[i]/100)*pm[i]['enthalphy']
  smix = -1*smix

  big,small = diff(els)
  # print("big atoms : ",big)
  # print("small atoms : ",small)



  rad = (comps(dcomp,big)-comps(dcomp,small))/(comps(dcomp,big))

  en = (electro(dcomp,big)-electro(dcomp,small))/(electro(dcomp,big)+electro(dcomp,small))


  return [hmix,smix,rad,en,els[np.argmax(comp)]]

In [ ]:
prepare_params('Mg61.5Cu29.5Gd9')

[10.115, 0.8758169838311579, 0.3151805999907267, -0.11152696133270965, 'Mg']

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Mendeley Dataset/New_Folder/balanced_da.csv")
data.head()

,Alloy_Formula,D_max,Phase
0,Al33.6Ca66.4,1.0,BMG
1,Cu46Zr54,2.0,BMG
2,Cu50Zr50,2.0,BMG
3,Cu64Zr36,2.0,BMG
4,Ni60Nb40,1.0,BMG


In [ ]:
j = 0
h_mix = []
s_mix = []
delta_d = []
delta_e = []
Dominant_element = []
for i in range(data.shape[0]):
  # print(i)
  # print(type(i))
  
  try:
    hmix,smix,rad,en,dom = prepare_params(data.iloc[i][0]);
    h_mix.append(hmix)
    s_mix.append(smix)
    delta_d.append(rad)
    delta_e.append(en)
    Dominant_element.append(dom)
  except:
    print("failed to create params : ",data.iloc[i][0])
    j = j + 1
print("Removed due to same atomic radius",j)


Removed due to same atomic radius 0


In [ ]:
data["h_mix"] = h_mix
data["s_mix"] = s_mix 
data["delta_d"] = delta_d
data["delta_e"] = delta_e
data["Dominant_element"] = Dominant_element

In [ ]:
data.head()

,Alloy_Formula,D_max,Phase,h_mix,s_mix,delta_d,delta_e,Dominant_element
0,Al33.6Ca66.4,1.0,BMG,9.25912,0.638347,0.203463,-0.233716,Ca
1,Cu46Zr54,2.0,BMG,17.36600,0.689944,0.247312,-0.176471,Zr
2,Cu50Zr50,2.0,BMG,17.05000,0.693147,0.247312,-0.176471,Cu
3,Cu64Zr36,2.0,BMG,15.94400,0.653418,0.247312,-0.176471,Cu
4,Ni60Nb40,1.0,BMG,21.04000,0.673012,0.212560,-0.088319,Ni
